In [417]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

sparkApp = SparkSession.builder.appName("Spark Task").getOrCreate()

In [418]:
df = sparkApp.read.csv("dataset.csv",header=True,inferSchema=True)

In [419]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Organization: string (nullable = true)
 |-- Skills: string (nullable = true)
 |-- Ratings: double (nullable = true)
 |-- Review counts: string (nullable = true)
 |-- Metadata: string (nullable = true)



In [420]:
df.describe()

DataFrame[summary: string, _c0: string, Title: string, Organization: string, Skills: string, Ratings: string, Review counts: string, Metadata: string]

In [421]:
df.show(10)

+---+--------------------+------------------+--------------------+-------+-----------------+--------------------+
|_c0|               Title|      Organization|              Skills|Ratings|    Review counts|            Metadata|
+---+--------------------+------------------+--------------------+-------+-----------------+--------------------+
|  0|Google Cybersecurity|            Google| Network Security...|    4.8| 4.8(20K reviews)|Beginner · Profes...|
|  1|Google Data Analy...|            Google| Data Analysis, R...|    4.8|4.8(137K reviews)|Beginner · Profes...|
|  2|Google Project Ma...|            Google| Project Manageme...|    4.8|4.8(100K reviews)|Beginner · Profes...|
|  3|    IBM Data Science|               IBM| Python Programmi...|    4.6|4.6(120K reviews)|Beginner · Profes...|
|  4|Google Digital Ma...|            Google| Digital Marketin...|    4.8| 4.8(23K reviews)|Beginner · Profes...|
|  5|    IBM Data Analyst|               IBM| Python Programmi...|    4.6| 4.6(73K revie

In [422]:
valuesOfMetadata = df.groupBy('Metadata').count().show(truncate=False)

+------------------------------------------------------+-----+
|Metadata                                              |count|
+------------------------------------------------------+-----+
|Beginner · Professional Certificate · 1 - 4 Weeks     |1    |
|Intermediate · Guided Project · Less Than 2 Hours     |5    |
|Beginner · Guided Project · Less Than 2 Hours         |14   |
|Beginner · Course · 1 - 3 Months                      |88   |
|Mixed · Course · 1 - 4 Weeks                          |13   |
|Intermediate · Course · 1 - 3 Months                  |43   |
|Intermediate · Specialization · 3 - 6 Months          |54   |
|Advanced · Course · 1 - 3 Months                      |6    |
|Beginner · Specialization · 1 - 3 Months              |17   |
|Mixed · Course · 1 - 3 Months                         |25   |
|Advanced · Specialization · 1 - 3 Months              |1    |
|Beginner · Course · 1 - 4 Weeks                       |89   |
|Beginner · Specialization · 3 - 6 Months              

In [423]:
#Selecting the columns needed for transformation
revCounts = df.select('Review counts')
metadata = df.select('Metadata')

In [424]:
#Transforming the Review counts Column
df = df.withColumn('Number of Reviews',((substring(df['Review counts'],5,2))*1000).cast("int"))

In [425]:
#Transforming the Metadata Column to create two new features (Difficulty - Duration (Min - Max))
difficultyRet = df.withColumn('Diff1',split(df['Metadata'],'·').getItem(0)).withColumn('Diff2',split(df['Metadata'],'·').getItem(1))

df = difficultyRet.withColumn('Difficulty',concat(col('Diff1'),lit('-'),col('Diff2'))).drop(*['Diff1','Diff2',])

df = df.withColumn('Duration',split(df['Metadata'],'·').getItem(2)).drop(*['Review counts','Metadata'])

df = df.withColumn('Min Duration',split(df['Duration'],'-').getItem(0).cast("int"))

maxDurationRet = df.withColumn('MaxDur TEMP',split(df['Duration'],'-').getItem(1))

df = maxDurationRet.withColumn('Max Duration',substring(maxDurationRet['MaxDur TEMP'],1,2).cast("int"))

df = df.withColumn('Unit',trim(substring(maxDurationRet['MaxDur TEMP'],3,8)))


In [426]:
#Creating the final, transformed Dataframe to carry out Analysis on
finalDF = df.drop(*['Duration','MaxDur TEMP'])

finalDF.show()

+---+--------------------+--------------------+--------------------+-------+-----------------+--------------------+------------+------------+------+
|_c0|               Title|        Organization|              Skills|Ratings|Number of Reviews|          Difficulty|Min Duration|Max Duration|  Unit|
+---+--------------------+--------------------+--------------------+-------+-----------------+--------------------+------------+------------+------+
|  0|Google Cybersecurity|              Google| Network Security...|    4.8|            20000|Beginner - Profes...|           3|           6|Months|
|  1|Google Data Analy...|              Google| Data Analysis, R...|    4.8|            13000|Beginner - Profes...|           3|           6|Months|
|  2|Google Project Ma...|              Google| Project Manageme...|    4.8|            10000|Beginner - Profes...|           3|           6|Months|
|  3|    IBM Data Science|                 IBM| Python Programmi...|    4.6|            12000|Beginner - P

In [427]:
finalDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Organization: string (nullable = true)
 |-- Skills: string (nullable = true)
 |-- Ratings: double (nullable = true)
 |-- Number of Reviews: integer (nullable = true)
 |-- Difficulty: string (nullable = true)
 |-- Min Duration: integer (nullable = true)
 |-- Max Duration: integer (nullable = true)
 |-- Unit: string (nullable = true)



In [428]:
TransCSV = finalDF.write.csv('TransCSV',header=True,mode="Overwrite")

In [429]:
#Number of Courses provided by a certain organization

numOfCoursesByOrg = finalDF\
    .groupBy('Organization')\
    .agg({'Organization':'Count'})\
    .orderBy('Organization')\
    .withColumnRenamed('count(Organization)','Number of Courses By Org')\
    .show()

+--------------------+------------------------+
|        Organization|Number of Courses By Org|
+--------------------+------------------------+
|Advancing Women i...|                       1|
|Akamai Technologi...|                       2|
|Alibaba Cloud Aca...|                       1|
| Amazon Web Services|                       7|
|Arizona State Uni...|                       3|
|Association of In...|                       1|
|           Atlassian|                       1|
|            Autodesk|                       1|
|             Berklee|                       3|
|California Instit...|                       3|
|         Campus BBVA|                       1|
|Cisco Learning an...|                       4|
|               Codio|                       1|
| Columbia University|                       3|
|             CompTIA|                       1|
|Copenhagen Busine...|                       2|
|Coursera Project ...|                      19|
|   Dartmouth College|                  

In [430]:
#Number of Courses provided by a certain organization with rating 4.5 and above

numOfCoursesByOrg = finalDF\
    .filter(col('Ratings')>4.5)\
    .groupBy('Organization')\
    .agg({'Organization':'Count'})\
    .orderBy('Organization')\
    .withColumnRenamed('count(Organization)','Number of Courses Rated 4.5+')\
    .show()

+--------------------+----------------------------+
|        Organization|Number of Courses Rated 4.5+|
+--------------------+----------------------------+
|Akamai Technologi...|                           2|
|Alibaba Cloud Aca...|                           1|
| Amazon Web Services|                           6|
|Arizona State Uni...|                           3|
|Association of In...|                           1|
|           Atlassian|                           1|
|            Autodesk|                           1|
|             Berklee|                           3|
|California Instit...|                           3|
|         Campus BBVA|                           1|
|Cisco Learning an...|                           3|
|               Codio|                           1|
| Columbia University|                           2|
|             CompTIA|                           1|
|Coursera Project ...|                           8|
|   Dartmouth College|                           1|
|Deep Teachi

In [431]:
#Average rating of courses provided by a certain Organization and Filtering with Organizations with Ratings above 4.5

avgRatingOfOrg = finalDF\
    .select('Organization','Ratings')\
    .groupBy('Organization')\
    .agg({'Ratings':'avg'})\
    .orderBy('Organization')\
    .withColumn('Average Rating',format_number(col('avg(Ratings)'),2))\
    .drop('avg(Ratings)')\
    .filter(col('Average Rating')>4.50)\
    .show()

+--------------------+--------------+
|        Organization|Average Rating|
+--------------------+--------------+
|Akamai Technologi...|          4.95|
|Alibaba Cloud Aca...|          4.60|
| Amazon Web Services|          4.76|
|Arizona State Uni...|          4.90|
|Association of In...|          4.60|
|           Atlassian|          4.70|
|            Autodesk|          4.70|
|             Berklee|          4.80|
|California Instit...|          4.70|
|         Campus BBVA|          4.70|
|Cisco Learning an...|          4.67|
|               Codio|          4.60|
| Columbia University|          4.70|
|             CompTIA|          4.60|
|   Dartmouth College|          4.60|
|Deep Teaching Sol...|          4.85|
|     DeepLearning.AI|          4.75|
|     Duke University|          4.57|
|          EC-Council|          4.55|
|EDHEC Business Sc...|          4.60|
+--------------------+--------------+
only showing top 20 rows



In [432]:
#Courses with ratings above 4.5 and over 50K Reviews

highRatedCourses = finalDF\
    .select('Title','Ratings','Number of Reviews')\
    .filter(col('Ratings')>4.50)\
    .filter(col('Number of Reviews')>50000)\
    .orderBy('Title')\
    .show()

+--------------------+-------+-----------------+
|               Title|Ratings|Number of Reviews|
+--------------------+-------+-----------------+
|(ISC)² Systems Se...|    4.7|            54000|
|API Security on G...|    4.6|            81000|
|Architecting with...|    4.7|            57000|
|Auditing I: Conce...|    4.7|            52000|
|Automate Cybersec...|    4.8|            91000|
|Blockchain Revolu...|    4.7|            93000|
|Business Transfor...|    4.6|            89000|
|Cloud Systems Sof...|    4.6|            77000|
|Create a Profile ...|    4.7|            71000|
|Cybersécurité Google|    4.7|            67000|
|Data Science Codi...|    4.7|            54000|
|Data Science Fund...|    4.6|            60000|
|Devenir entrepren...|    4.7|            59000|
|Excel Skills for ...|    4.9|            59000|
|Faster Together, ...|    4.8|            97000|
|FinTech Law and P...|    4.8|            81000|
|FinTech: Finance ...|    4.7|            77000|
|Finance for Techn..

In [433]:
#Courses suitable for Beginners
beginnerCourses = finalDF\
    .select('Title','Difficulty')\
    .filter(col('Difficulty')\
    .startswith("Beginner"))\
    .show(truncate=False)
#Number of them
numOfBeginnerCourses = finalDF\
    .select('Title','Difficulty')\
    .filter(col('Difficulty')\
    .startswith("Beginner"))\
    .count()
print(f"Number of courses suitable for beginners: {numOfBeginnerCourses}")



+---------------------------------------------+------------------------------------+
|Title                                        |Difficulty                          |
+---------------------------------------------+------------------------------------+
|Google Cybersecurity                         |Beginner - Professional Certificate |
|Google Data Analytics                        |Beginner - Professional Certificate |
|Google Project Management:                   |Beginner - Professional Certificate |
|IBM Data Science                             |Beginner - Professional Certificate |
|Google Digital Marketing & E-commerce        |Beginner - Professional Certificate |
|IBM Data Analyst                             |Beginner - Professional Certificate |
|Google IT Support                            |Beginner - Professional Certificate |
|Machine Learning                             |Beginner - Specialization           |
|Google UX Design                             |Beginner - Profess

In [438]:
#List of Courses that would take weeks to complete
shortCourses = finalDF\
    .select('Title','Organization','Min Duration','Max Duration','Unit')\
    .filter(col('Unit')=='Weeks')\
    .show(truncate=False)

+----------------------------------------------------------+---------------+------------+------------+-----+
|Title                                                     |Organization   |Min Duration|Max Duration|Unit |
+----------------------------------------------------------+---------------+------------+------------+-----+
|Generative AI with Large Language Models                  |DeepLearning.AI|1           |4           |Weeks|
|Generative AI for Everyone                                |DeepLearning.AI|1           |4           |Weeks|
|Foundations of Cybersecurity                              |Google         |1           |4           |Weeks|
|Foundations of Project Management                         |Google         |1           |4           |Weeks|
|Supervised Machine Learning: Regression and Classification|DeepLearning.AI|1           |4           |Weeks|
|AI For Everyone                                           |DeepLearning.AI|1           |4           |Weeks|
|Foundations of Dig

In [435]:
#Creating a Temporary View to carry out SparkSQL Queries on
finalDF.createOrReplaceTempView("analysis")

In [436]:
#Retrieving all High Rated Data-Related Courses

ratedDataCourses = sparkApp.sql("SELECT Title AS RatedDataCourses, Ratings FROM analysis WHERE Title LIKE '%Data%' AND Ratings >= 4.5 ORDER BY Ratings DESC;")

ratedDataCourses.show(truncate=False)

+-------------------------------------------------------------------------+-------+
|RatedDataCourses                                                         |Ratings|
+-------------------------------------------------------------------------+-------+
|ChatGPT Advanced Data Analysis                                           |4.9    |
|Python Data Structures                                                   |4.9    |
|Data Visualization with Advanced Excel                                   |4.8    |
|Foundations: Data, Data, Everywhere                                      |4.8    |
|Google Data Analytics                                                    |4.8    |
|Introduction to Data Analytics                                           |4.8    |
|Process Data from Dirty to Clean                                         |4.8    |
|Data Analysis with R Programming                                         |4.8    |
|Excel Basics for Data Analysis                                           |4

In [437]:
#Courses with Low Ratings
badRatedCourses = sparkApp.sql('SELECT Organization,Title as CourseName, Ratings, `Number of Reviews` FROM analysis ORDER BY Ratings ASC LIMIT 10;')

badRatedCourses.show(truncate=False)

+--------------------------+--------------------------------------------+-------+-----------------+
|Organization              |CourseName                                  |Ratings|Number of Reviews|
+--------------------------+--------------------------------------------+-------+-----------------+
|Johns Hopkins University  |GPU Programming                             |2.8    |72000            |
|Microsoft                 |Perform data science with Azure Databricks  |3.4    |44000            |
|Fractal Analytics         |Python for Data Science                     |3.7    |29000            |
|University of Pennsylvania|Fundamentals of Finance                     |3.9    |84000            |
|Howard University         |Linear Algebra for Data Science Using Python|3.9    |28000            |
|Coursera Project Network  |Statistics For Data Science                 |4.0    |19000            |
|Duke University           |MLOps | Machine Learning Operations         |4.0    |15000            |
